# Regression Models Analysis on House Sales Data

# Business Problem

ABC company intereste to buy property in King County. They are new to the area, and interest about the housing in Seattle. They want to decide if they should buy property in Seattle or other area within King County.

The goal is to use regression models to predict sale price and translate these findings into actionable insights for ABC company.

Library that are used for the data analysis

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
%matplotlib inline

# scaling and train test split
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import MinMaxScaler

#one hot encoding
from sklearn.preprocessing import OneHotEncoder
#linear regression model
from sklearn.linear_model import LinearRegression
#evalutate multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
# for QQ plot
import scipy.stats as stats
# evaluation on test data
from sklearn import metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
from sklearn.metrics import classification_report,confusion_matrix

King County House Sales dataset


This data set can be found in kc_house_data.csv in the following GitHub repository
https://github.com/learn-co-curriculum/dsc-phase-2-project-v2-3

In [ ]:
# will use this table to do regression analysis to analyze King county house sales and predict house sale price
file_path = "\\Users\\eggfr\\Flatiron\\Flatiron_phase2_project\\dsc-phase-2-project\\data\\kc_house_data.csv"
project2_raw_df = pd.read_csv(file_path)

In [ ]:
plt.figure(figsize=(20,8))
sns.heatmap(project2_raw_df.corr(),annot = True)

Column Names and Description

id: Unique ID for each home sold
date: Date of the home sale
price: Price of each home sold
bedrooms: Number of bedrooms
bathrooms: Number of bathrooms, where .5 accounts for a room with a toilet but no shower
sqft_living: Square footage of the apartments interior living space
sqft_lot: Square footage of the land space
floors: Number of floors
waterfront: - A dummy variable for whether the apartment was overlooking the waterfront or not
view: An index from 0 to 4 of how good the view of the property was
condition: - An index from 1 to 5 on the condition of the apartment,
grade: An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
sqft_above: The square footage of the interior housing space that is above ground level
sqft_basement: The square footage of the interior housing space that is below ground level
yr_built: The year the house was initially built
yr_renovated: The year of the house’s last renovation
zipcode: What zipcode area the house is in
lat: Lattitude
long: Longitude
sqft_living15: The square footage of interior housing living space for the nearest 15 neighbors
sqft_lot15: The square footage of the land lots of the nearest 15 neighbors

In [ ]:
project2_raw_df.head()

In [ ]:
y = project2_raw_df['price']
X= project2_raw_df.drop('price',1)
X.shape

# Train test split

Separating data into training and testing sets is an important part of evaluating the models.Most of the data is used for training, and a smaller portion of the data is used for testing. For this analysis: we only split data into train and test. 75% of the data is for training and 25% for test. Also, the data split happened before we even do any EDA analysis to prevent data leakage. There is 16197 row of datas for the train set and 5400 rows of the data for test set before any data cleaning or analysis is done.


In [ ]:
#create train-test set using 75%-25% ratio for the train set and test set and set the random state = 42) randomly split the data
x_train, x_test, y_train, y_test = train_test_split(X, y ,test_size=0.25,random_state=42)
# shape of train and test splits
x_train.shape, x_test.shape, y_train.shape, y_test.shape

# Data cleaning

# Checking category type for each column

OLS analysis only take data type in numerical form (int64 or float64). We need to change the categotical data into numerical form. Also, we need to check missing data for each column. The section below shows there is missing values on the category of waterfront, view, and year renovated. We will go through every category and clean the data as necessary before OLS analysis.

We need to check missing value and data type. We need to identify categorical nominal variable and categotical oridnal variable. For categorical nominal variable, we need to transform it to numerical form using One Hot Encdoing method. No adjustment is needed for categotical ordinal variable.

Also, We have to make changes for both sets since we split our data into train set and test set. Otherwise, the analysis will be incorrect.

Checking datatype of the data

In [ ]:
project2_raw_df.info()

Checking missing value of the data

In [ ]:
# determine count of missing values in table
project2_raw_df.isnull().sum()

Preprocessing date-month-year

In [ ]:
#Split date, year, month into seperate columns
x_train['date'] = pd.to_datetime(x_train['date'])
x_train['month'] = x_train['date'].apply(lambda date:date.month)
x_train['year'] = x_train['date'].apply(lambda date:date.year)
x_train = x_train.drop('date',axis=1)

x_test['date'] = pd.to_datetime(x_test['date'])
x_test['month'] = x_test['date'].apply(lambda date:date.month)
x_test['year'] = x_test['date'].apply(lambda date:date.year)
x_test = x_test.drop('date',axis=1)


Preprocessing sqft_basement

There is missing value in "sqft_basement" variable. I will replace the 1 missing value with the mean of the value, which is 297.

In [ ]:
#checking unique category for column ('sqft_basement'), there is a datadype'?''.we replace it withe mean of value.
project2_raw_df['sqft_basement'].unique()

In [ ]:
# change the inproper datatype for 'sqft_basement' and replace it withe mean of value, 297.
x_train['sqft_basement'] = x_train['sqft_basement'].apply(lambda x: float(x.replace("?", "297")))
x_train['sqft_basement'] = x_train['sqft_basement'].apply(lambda x: float(x))
#x_train['sqft_basement'].dtype
x_test['sqft_basement'] = x_test['sqft_basement'].apply(lambda x: float(x.replace("?", "297")))
x_test['sqft_basement'] = x_test['sqft_basement'].apply(lambda x: float(x))



Preprocessing floors

Checking the category type for variable "floors". No action is needed as it is categorical ordinal variable.

In [ ]:
#check the category of floors- categorical ordinal variable. No adjustment is needed.
project2_raw_df['floors'].value_counts()

Preprocessing waterfront

There is missing value in "waterfront" variable, and majority of the "waterfront" is None. I will replace the missing values with No data type. Also, waterfront is also a categorical variable, it will be transformed into 1 for yes and 0 for no.

In [ ]:
# percentage of waterfront in the sale data.
project2_raw_df['waterfront'].value_counts(normalize = True)

In [ ]:
#fill na with no value and change yes to 1 n no to 0 #since majority of the sales has no waterfront.
x_train['waterfront'] = x_train['waterfront'].fillna(value = 'NO')
x_train['waterfront'] = x_train['waterfront'].replace(to_replace = ['YES','NO'],value = [1,0])
x_train['waterfront'].value_counts()


In [ ]:
x_test['waterfront'] = x_test['waterfront'].fillna(value = 'NO')
x_test['waterfront'] = x_test['waterfront'].replace(to_replace = ['YES','NO'],value = [1,0])
x_test['waterfront'].value_counts()

Preprocessing view

There is missing value in "view" variable, and majority of the "view" is None. I will replace the missing values with None data type. Also, view is also a categorical variable, it will be transformed into 0 to 4 with 0 as none and 4 as excellent

In [ ]:
project2_raw_df['view'].value_counts(normalize = True)


In [ ]:
# fill na with no value -->majority is none and could be N/A as none as well. n change the catgorical ordinal in to 0 to 4 with 0 to none and 4 to excellent
x_train['view'] = x_train['view'].fillna(value = 'NONE')
x_test['view'] = x_test['view'].fillna(value = 'NONE')
x_train['view'] = x_train['view'].replace(to_replace = ['NONE','AVERAGE','GOOD','FAIR','EXCELLENT'],value = [0,1,2,3,4])
x_test['view'] = x_test['view'].replace(to_replace = ['NONE','AVERAGE','GOOD','FAIR','EXCELLENT'],value = [0,1,2,3,4])

Preprocessing Condition

There is no missing value on 'condition' variable, so no replacement is needed. 'Condition' is a categorical ordinal variable, it will be transformed into 0 to 4 with 0 as poor and 4 as very good.

In [ ]:
project2_raw_df['condition'].value_counts()

In [ ]:
# replace cateogrical rating with (0 to 4 scale. 0 - Poor, and 4 - Very Good)
x_train['condition'] = x_train['condition'].replace(to_replace = ['Poor','Fair','Average','Good','Very Good'],value = [0,1,2,3,4])
x_test['condition'] = x_test['condition'].replace(to_replace = ['Poor','Fair','Average','Good','Very Good'],value = [0,1,2,3,4])


Preprocessing Grade and create a new_grade column

There is no missing value on 'grade' variable, so no replacement is needed. However, a "new_grade" column is created and stored all the numerical value for the grade. As given from the data set, graded are scaled from 3 to 13, poor to mension.

In [ ]:
project2_raw_df['grade'].unique()

In [ ]:
# grab the numerical rating and assign it to interger type
x_train['new_grade'] = x_train['grade'].astype(str).str[0]
x_train['new_grade'] = x_train['new_grade'].astype(int)
x_test['new_grade'] = x_test['grade'].astype(str).str[0]
x_test['new_grade'] = x_test['new_grade'].astype(int)


In [ ]:
#drop the string grade column
x_train = x_train.drop(columns='grade')
x_test = x_test.drop(columns='grade')

Preprocessing bedrooms

Checking the category type for variable "bedrooms". No action is needed as it is categorical ordinal variable.

In [ ]:
project2_raw_df['bedrooms'].value_counts()

Preprocessing bathrooms

Checking the category type for variable "bathrooms". No action is needed as it is categorical ordinal variable.

In [ ]:
project2_raw_df['bathrooms'].value_counts()

Set up a new group "seattle"

Going to create a new column as seattle by zipcode which it solely belongs https://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?FIPS=53033 

In [ ]:
#grouping seattle into zipcode which it solely belongs https://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?FIPS=53033 
x_train['seattle'] = x_train['zipcode'].apply(lambda x : 0 if x == 98101 or x == 98102 or x == 98103 or x == 98104 or x == 98105 or x == 98106 or x == 98107 or x == 98109 or x == 98111 or x == 98112 or x == 98114 or x == 98116 or x == 98117 or x == 98118 or x == 98119 or x == 98121 or x == 98122 or x == 98124 or x == 98125 or x == 98126 or x == 98131 or x == 98132 or x == 98133 or x == 98134 or x == 98136 or x == 98144 or x == 98145 or x == 98146 or x == 98148 or x == 98154  or x == 98160 or x == 98161 or x == 98164 or x == 98166 or x == 98171 or x == 98174 or x== 98178 or x == 98199 else 1) 
x_test['seattle'] = x_test['zipcode'].apply(lambda x : 0 if x == 98101 or x == 98102 or x == 98103 or x == 98104 or x == 98105 or x == 98106 or x == 98107 or x == 98109 or x == 98111 or x == 98112 or x == 98114 or x == 98116 or x == 98117 or x == 98118 or x == 98119 or x == 98121 or x == 98122 or x == 98124 or x == 98125 or x == 98126 or x == 98131 or x == 98132 or x == 98133 or x == 98134 or x == 98136 or x == 98144 or x == 98145 or x == 98146 or x == 98148 or x == 98154  or x == 98160 or x == 98161 or x == 98164 or x == 98166 or x == 98171 or x == 98174 or x== 98178 or x == 98199 else 1) 


Preprocessing zipcode via One Hot Encoding

While zipcode is numerical variable in the data table, it is categorical nominal variable. Different zipcode means different category. We are going to transform it into a numerical variable via One Hot Encoding.

In [ ]:
#onehot Econdinf zipcode for x_train
ohe = OneHotEncoder(drop='first')
data =  x_train[['zipcode']]

view_df = pd.DataFrame(ohe.fit_transform(data).toarray(), index = x_train.index) #index = x_train.index to match x_train index for concat later
view_df.columns = ohe.get_feature_names()   #use get_feature_names() to get feature name back after one hot encoding

x_train = pd.concat([x_train,view_df],axis=1)
x_train.shape

In [ ]:
#onehot Econdinf zipcode for x_test
ohe = OneHotEncoder(drop='first')
data =  x_test[['zipcode']]

view_df = pd.DataFrame(ohe.fit_transform(data).toarray(), index = x_test.index) #index = x_train.index to match x_train index for concat later
view_df.columns = ohe.get_feature_names()   #use get_feature_names() to get feature name back after one hot encoding

x_test = pd.concat([x_test,view_df],axis=1)
x_test.shape

Preprocessing yr_renovated and create is_renovated

yr_renovated is an indication if the house is renovated. There is 0.0 in the value counts and that shows there is a category that house that is not renovated. With majority of the sale is not renovated. I would assume the missing value is just not renovated, and filled the missing value with 0. Also, I also change this to a categoty variable with 0 = not renovated and 1 is renovated and stored it in a var variable, is_renovated.

In [ ]:
project2_raw_df['yr_renovated'].value_counts()

In [ ]:
# fillna with 0 to NaN for year_renovated- assuming there is no renovation.
x_train['yr_renovated'] = x_train['yr_renovated'].fillna(value = 0)
x_test['yr_renovated'] = x_test['yr_renovated'].fillna(value = 0)

In [ ]:
x_train['is_renovated'] = x_train['yr_renovated'].apply(lambda x: 0 if x ==0 else 1)
x_test['is_renovated'] = x_test['yr_renovated'].apply(lambda x: 0 if x ==0 else 1)
#x_test['sqft_basement'] = x_test['sqft_basement'].apply(lambda x: float(x.replace("?", "297")))

# Log normal the price

Price look positively skewed with a right tail, and we may have to account for outliers later on of the analysis. For now, lets log the price so we have amore normalized distribution.

In [ ]:
p = sns.kdeplot(y_train)
p.set( title = "Distribution of Home Sale Price")

In [ ]:
#take a log on price
y_train = np.log(y_train)
y_test = np.log(y_test)


In [ ]:
p = sns.kdeplot(y_train)
p.set( title = "Distribution of Home Sale Price")

Recheck our x_train data type and missing value.

Let's recheck our data type to see if everything is numerical form and also check if we have any missing data.

In [ ]:
x_train.info()

Every category is non-null and is in either int64 or float64.

scale data w MinMaxScaler

Since 'sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15' are measured in different units, they will be scaled with MinMaxScaler

In [ ]:
#scale data w MinMaxScaler 'sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15 for both train n data set
features = ['sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15']
autoscaler = MinMaxScaler()

x_train[features] = autoscaler.fit_transform(x_train[features])
x_test[features] = autoscaler.fit_transform(x_test[features])

Dropping some columns that are not useful for the analysis

In [ ]:
x_train = x_train.drop(columns=['id','year','zipcode'])
x_test = x_test.drop(columns=['id','year','zipcode'])

In [ ]:
x_test.shape

# Baseline Model (with every predictors from the data set)

I decided to use every predictors from the data as my baseline model and just to get a feeling on how this would work with linear regression.

In [ ]:
import statsmodels.api as sm
y = y_train
X= x_train
z = x_test
Hh = y_test
Xcont = sm.add_constant(X)

model = sm.OLS(endog = y, exog = Xcont)
model = sm.OLS(y,Xcont)
res = model.fit()
print(res.summary())

In [ ]:
ols = LinearRegression()

testsmodel = ols.fit(X,y)
print(testsmodel.score(X,y)) # train
print(testsmodel.score(z,Hh)) # test 
#print(cross_val_score(lr,X,y,cv=3))

Our baseline R2 is at 0.87. However, I am using every predictors that could be found in the dataset to achieve this. Obviously, most of our variance can be explained with every predictors as inputs.

We'll use this as a baseline moving forward and see if we can keep our R2 score close to 0.87

The Training and Test scores are with 5%, so it shows me that the our model is not underfit/overfit.

In [ ]:
model1_predictions = ols.predict(x_test)

In [ ]:
fig = plt.figure()
plt.scatter(np.exp(model1_predictions), np.exp(y_test))
plt.xlabel('predicting testing price in $')
plt.ylabel('actual testing sale price in $')
fig.suptitle('prediction price vs actual sale price')


Check the Normality Assumption

Q-Q plot is a scatterplot created by plotting two sets of quantiles against one another. If both sets of quantiles came from the same distribution, we should see the points forming a line that’s roughly straight.. We are plotting our model residual and normal data. There is a tail on the left hand side, and we have to eliminate that.

In [ ]:
fig = sm.graphics.qqplot(res.resid, dist=stats.norm, line='45', fit=True)
ax.set_title("QQ-plot")

Baseline model residual plot

Clear underestimate when the sales price increase.Further subsetting is potentially warranted

In [ ]:
plt.scatter(model1_predictions,(y_test-model1_predictions))
plt.axhline(y=0, color='r', linestyle='-')
plt.title('baseline residual plot')
plt.show()

# 2nd Model. Feature enginering with the investigation of Multicollinearity via VIF test.

All predictors are used in the baselinVe model.There are some chances that the baseline has strong multicollinearity issues. Varince_inflation_factor is used to feature select some predictors.
When WIF score is higher than 5, we consider it has some collinearity issue with other predicors. In this model, we will remove features that has VIF score higher than 5

In [ ]:
#perform vif_test
x_train_list= list(x_train.columns.values)
feature = x_train_list
X = x_train[feature]
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_list = list(zip(feature, vif))
vif_scores = list(zip(feature, vif))
x_cols = [x for x,vif in vif_scores if vif < 5]
print(len(vif_scores), len(x_cols))


We reduce our predictors from 89 to 35 after the VIF test

In [ ]:
y = y_train
Hh = y_test
feature = x_cols
#feature = ['bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15', 'month', 'new_grade', 'x0_98002', 'x0_98003', 'x0_98005', 'x0_98007', 'x0_98010', 'x0_98022', 'x0_98023', 'x0_98024', 'x0_98030', 'x0_98031', 'x0_98032', 'x0_98039', 'x0_98040', 'x0_98042', 'x0_98055', 'x0_98056', 'x0_98058', 'x0_98070', 'x0_98092', 'x0_98108', 'x0_98168', 'x0_98188', 'x0_98198']
#feature = ['bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15', 'month','new_grade']

X= x_train[feature]
z= x_test[feature]
Xcont = sm.add_constant(X)

model = sm.OLS(endog = y, exog = Xcont)
model = sm.OLS(y,Xcont)
res = model.fit()
print(res.summary())

In [ ]:
ols2 = LinearRegression()
testsmodel = ols2.fit(X,y)
print(testsmodel.score(X,y)) # train
print(testsmodel.score(z,Hh)) # test

In [ ]:
model2_predictions = ols2.predict(z)

Model2 R2 is at 0.69. However, and some predictors is eliminated while R2 score decreases. 

The Training and Test scores are with 3%, so it shows me that the our model is not underfit/overfit.



There is outliers that should be eliminated and see how that affect the model if outliers is eliminated in later models.

In [ ]:
fig = plt.figure()
plt.scatter(np.exp(model2_predictions), np.exp(y_test))
plt.xlabel('sale price in $')
plt.xlabel('predicting testing price in $')
plt.ylabel('actual testing sale price in $')
fig.suptitle('prediction price vs actual sale price')

QQ-plot : points forming a line that’s roughly straight line- left tail is reducing.

In [ ]:
fig = sm.graphics.qqplot(res.resid, dist=stats.norm, line='45', fit=True)
ax.set_title("QQ-plot")

In [ ]:
plt.scatter(np.exp(model2_predictions),(np.exp(y_test)-np.exp(model2_predictions)))
plt.axhline(y=0, color='r', linestyle='-')
plt.title("model2 residual plot")
plt.show()

# 3rd Model. Feature enginering with eliminating outliers from model 2 and subsetting some inputs.

As working from the 2nd model- I see some outliers from the sale price- Lets take a look on the historgram from the original data. I am going to use 97% percentile as our subsetting range for price.

project2_raw_df.price.hist()

In [ ]:
for i in range(90, 99):
    q = i / 100
    print('{} percentile: {}'.format(q, project2_raw_df['price'].quantile(q=q)))

Also, I am looking at the highest coefficient from the 2nd model, and reduce outliers from the ones that have high coefficeient. The categories that will be removed some outliers as we follow.
sqft_lot is using 0.95 percentile, which is 43307
sqft_living is using 0.96 percentile, which is 3920
Bedrooms category is subsetting to 12 to remove the 33 bedrooom outlier
Bathrooms category is subsetting to 6 

In [ ]:
for i in range(90, 99):
    q = i / 100
    print('{} percentile: {}'.format(q, project2_raw_df['sqft_lot'].quantile(q=q)))

In [ ]:
for i in range(90, 99):
    q = i / 100
    print('{} percentile: {}'.format(q, project2_raw_df['sqft_living'].quantile(q=q)))

As we there is 33 bedrooms in our dataset, I would consider that as an outlier and drop it from our analysis.

In [ ]:
project2_raw_df['bedrooms'].value_counts()

In [ ]:
project2_raw_df['bathrooms'].value_counts()

In [ ]:
rem_project2_raw_df =project2_raw_df
rem_project2_raw_df.shape

In [ ]:
rem_project2_raw_df = rem_project2_raw_df.loc[rem_project2_raw_df['price']< 11600000]
rem_project2_raw_df = rem_project2_raw_df.loc[rem_project2_raw_df['bathrooms']< 6]
rem_project2_raw_df = rem_project2_raw_df.loc[rem_project2_raw_df['bedrooms']< 12]
rem_project2_raw_df = rem_project2_raw_df.loc[rem_project2_raw_df['sqft_living']< 3920]
rem_project2_raw_df = rem_project2_raw_df.loc[rem_project2_raw_df['sqft_lot']< 43307]
rem_project2_raw_df.shape

In [ ]:
rem_project2_raw_df.price.hist()
plt.title('price histogram')
plt.xlabel('sale price')

In [ ]:
rem_project2_raw_df.bedrooms.hist()
plt.title('bedroom histogram')
plt.xlabel('number of bedroom')

In [ ]:
rem_project2_raw_df.sqft_living.hist()
plt.title('sqft_living histogram')
plt.xlabel('sqft_living')

In [ ]:
rem_project2_raw_df.sqft_lot.hist()
plt.title('sqft_lot histogram')
plt.xlabel('sqft_lot')

Re-create the data set for this model.

Since I add the conditions for several predictors, the size of the original x_train,x_test data set is going to be different. Hence, I need to recreate the x_train as x_train1, x_test as x_test1, y_train as y_train1, and y_test1. Also, two more cities categories are created in this so "seattle" data can be compated. The data set is dropped from 21597 to 19880.

In [ ]:

rem_y = rem_project2_raw_df['price']
rem_X= rem_project2_raw_df.drop('price',1)
rem_X.shape

In [ ]:
rem_y = rem_project2_raw_df['price']
rem_X= rem_project2_raw_df.drop('price',1)

In [ ]:
#create train-test set using 75-25 (train-test and random state = 42) randomly split the data
x_train1, x_test1, y_train1, y_test1 = train_test_split(rem_X, rem_y ,test_size=0.25,random_state=42)
# shape of train and test splits
x_train1.shape, x_test1.shape, y_train1.shape, y_test1.shape

In [ ]:
rem_project2_raw_df.info()

In [ ]:
rem_project2_raw_df.isnull().sum()

In [ ]:
rem_project2_raw_df['seattle'] = rem_project2_raw_df['zipcode'].apply(lambda x : 0 if x == 98101 or x == 98102 or x == 98103 or x == 98104 or x == 98105 or x == 98106 or x == 98107 or x == 98109 or x == 98111 or x == 98112 or x == 98114 or x == 98116 or x == 98117 or x == 98118 or x == 98119 or x == 98121 or x == 98122 or x == 98124 or x == 98125 or x == 98126 or x == 98131 or x == 98132 or x == 98133 or x == 98134 or x == 98136 or x == 98144 or x == 98145 or x == 98146 or x == 98148 or x == 98154  or x == 98160 or x == 98161 or x == 98164 or x == 98166 or x == 98171 or x == 98174 or x== 98178 or x == 98199 else 1) 
rem_project2_raw_df['seattle'] = rem_project2_raw_df['zipcode'].apply(lambda x : 0 if x == 98101 or x == 98102 or x == 98103 or x == 98104 or x == 98105 or x == 98106 or x == 98107 or x == 98109 or x == 98111 or x == 98112 or x == 98114 or x == 98116 or x == 98117 or x == 98118 or x == 98119 or x == 98121 or x == 98122 or x == 98124 or x == 98125 or x == 98126 or x == 98131 or x == 98132 or x == 98133 or x == 98134 or x == 98136 or x == 98144 or x == 98145 or x == 98146 or x == 98148 or x == 98154  or x == 98160 or x == 98161 or x == 98164 or x == 98166 or x == 98171 or x == 98174 or x== 98178 or x == 98199 else 1) 
rem_project2_raw_df['kent'] = rem_project2_raw_df['zipcode'].apply(lambda x : 0 if x == 98030 or x == 98031 or x == 98032 or x == 98035 or x == 98042 or x == 98064 else 1)
rem_project2_raw_df['kent'] = rem_project2_raw_df['zipcode'].apply(lambda x : 0 if x == 98030 or x == 98031 or x == 98032 or x == 98035 or x == 98042 or x == 98064 else 1)
rem_project2_raw_df['bellevue'] = rem_project2_raw_df['zipcode'].apply(lambda x : 0 if x == 98004 or x == 98005 or x == 98006 or x == 98007 or x == 98008 or x == 98009 or x == 98015 else 1)
rem_project2_raw_df['bellevue'] = rem_project2_raw_df['zipcode'].apply(lambda x : 0 if x == 98004 or x == 98005 or x == 98006 or x == 98007 or x == 98008 or x == 98009 or x == 98015 else 1)

In [ ]:
rem_project2_raw_df.head()

In [ ]:
x_train1['date'] = pd.to_datetime(x_train1['date'])
x_train1['month'] = x_train1['date'].apply(lambda date:date.month)
x_train1['year'] = x_train1['date'].apply(lambda date:date.year)
x_train1 = x_train1.drop('date',axis=1)

x_test1['date'] = pd.to_datetime(x_test1['date'])
x_test1['month'] = x_test1['date'].apply(lambda date:date.month)
x_test1['year'] = x_test1['date'].apply(lambda date:date.year)
x_test1 = x_test1.drop('date',axis=1)


In [ ]:
# change the inproper datatype for 'sqft_basement'
x_train1['sqft_basement'] = x_train1['sqft_basement'].apply(lambda x: float(x.replace("?", "297")))
x_train1['sqft_basement'] = x_train1['sqft_basement'].apply(lambda x: float(x))
#x_train['sqft_basement'].dtype
x_test1['sqft_basement'] = x_test1['sqft_basement'].apply(lambda x: float(x.replace("?", "297")))
x_test1['sqft_basement'] = x_test1['sqft_basement'].apply(lambda x: float(x))

In [ ]:
x_train1['waterfront'] = x_train1['waterfront'].fillna(value = 'NO')
x_train1['waterfront'] = x_train1['waterfront'].replace(to_replace = ['YES','NO'],value = [1,0])
#x_train1['waterfront'].value_counts()
x_test1['waterfront'] = x_test1['waterfront'].fillna(value = 'NO')
x_test1['waterfront'] = x_test1['waterfront'].replace(to_replace = ['YES','NO'],value = [1,0])
#x_test1['waterfront'].value_counts()

In [ ]:
# fill na with no value -->majority is none and could be N/A as none as well. n change the catgorical ordinal in to 0 to 4 with 0 to none and 4 to excellent
x_train1['view'] = x_train1['view'].fillna(value = 'NONE')
x_test1['view'] = x_test1['view'].fillna(value = 'NONE')
x_train1['view'] = x_train1['view'].replace(to_replace = ['NONE','AVERAGE','GOOD','FAIR','EXCELLENT'],value = [0,1,2,3,4])
x_test1['view'] = x_test1['view'].replace(to_replace = ['NONE','AVERAGE','GOOD','FAIR','EXCELLENT'],value = [0,1,2,3,4])

In [ ]:
# replace cateogrical rating with (0 to 5 scale. 0 - Poor, and 4 - Very Good)
x_train1['condition'] = x_train1['condition'].replace(to_replace = ['Poor','Fair','Average','Good','Very Good'],value = [0,1,2,3,4])
x_test1['condition'] = x_test1['condition'].replace(to_replace = ['Poor','Fair','Average','Good','Very Good'],value = [0,1,2,3,4])


In [ ]:
# grab the numerical rating and assign it to interger type
x_train1['new_grade'] = x_train1['grade'].astype(str).str[0]
x_train1['new_grade'] = x_train1['new_grade'].astype(int)
x_test1['new_grade'] = x_test1['grade'].astype(str).str[0]
x_test1['new_grade'] = x_test1['new_grade'].astype(int)


In [ ]:
#drop the string grade column
x_train1 = x_train1.drop(columns='grade')
x_test1 = x_test1.drop(columns='grade')


In [ ]:
#grouping seattle into zipcode which it solely belongs https://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?FIPS=53033 
x_train1['seattle'] = x_train1['zipcode'].apply(lambda x : 1 if x == 98101 or x == 98102 or x == 98103 or x == 98104 or x == 98105 or x == 98106 or x == 98107 or x == 98109 or x == 98111 or x == 98112 or x == 98114 or x == 98116 or x == 98117 or x == 98118 or x == 98119 or x == 98121 or x == 98122 or x == 98124 or x == 98125 or x == 98126 or x == 98131 or x == 98132 or x == 98133 or x == 98134 or x == 98136 or x == 98144 or x == 98145 or x == 98146 or x == 98148 or x == 98154  or x == 98160 or x == 98161 or x == 98164 or x == 98166 or x == 98171 or x == 98174 or x== 98178 or x == 98199 else 0) 
x_test1['seattle'] = x_test1['zipcode'].apply(lambda x : 1 if x == 98101 or x == 98102 or x == 98103 or x == 98104 or x == 98105 or x == 98106 or x == 98107 or x == 98109 or x == 98111 or x == 98112 or x == 98114 or x == 98116 or x == 98117 or x == 98118 or x == 98119 or x == 98121 or x == 98122 or x == 98124 or x == 98125 or x == 98126 or x == 98131 or x == 98132 or x == 98133 or x == 98134 or x == 98136 or x == 98144 or x == 98145 or x == 98146 or x == 98148 or x == 98154  or x == 98160 or x == 98161 or x == 98164 or x == 98166 or x == 98171 or x == 98174 or x== 98178 or x == 98199 else 0) 
x_train1['kent'] = x_train1['zipcode'].apply(lambda x : 1 if x == 98030 or x == 98031 or x == 98032 or x == 98035 or x == 98042 or x == 98064 else 0)
x_test1['kent'] = x_test1['zipcode'].apply(lambda x : 1 if x == 98030 or x == 98031 or x == 98032 or x == 98035 or x == 98042 or x == 98064 else 0)
x_train1['bellevue'] = x_train1['zipcode'].apply(lambda x : 1 if x == 98004 or x == 98005 or x == 98006 or x == 98007 or x == 98008 or x == 98009 or x == 98015 else 0)
x_test1['bellevue'] = x_test1['zipcode'].apply(lambda x : 1 if x == 98004 or x == 98005 or x == 98006 or x == 98007 or x == 98008 or x == 98009 or x == 98015 else 0)

In [ ]:
x_train1['seattle'].mean()

In [ ]:
x_train1.info()

In [ ]:
#onehot Econdinf zipcode for x1_train
ohe = OneHotEncoder(drop='first')
data =  x_train1[['zipcode']]

view_df = pd.DataFrame(ohe.fit_transform(data).toarray(), index = x_train1.index) #index = x_train.index to match x_train index for concat later
view_df.columns = ohe.get_feature_names()   #use get_feature_names() to get feature name back after one hot encoding

x_train1 = pd.concat([x_train1,view_df],axis=1)
x_train1.shape

In [ ]:
#onehot Econdinf zipcode for x_test
ohe = OneHotEncoder(drop='first')
data =  x_test1[['zipcode']]

view_df = pd.DataFrame(ohe.fit_transform(data).toarray(), index = x_test1.index) #index = x_train.index to match x_train index for concat later
view_df.columns = ohe.get_feature_names()   #use get_feature_names() to get feature name back after one hot encoding

x_test1 = pd.concat([x_test1,view_df],axis=1)



In [ ]:
# fillna with 0 to NaN for year_renovated- assuming there is no renovation.
x_train1['yr_renovated'] = x_train1['yr_renovated'].fillna(value = 0)
x_test1['yr_renovated'] = x_test1['yr_renovated'].fillna(value = 0)

In [ ]:
#take a log on price
y_train1 = np.log(y_train1)
y_test1 = np.log(y_test1)

In [ ]:
#scale data w MinMaxScaler 'sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15 for both train n data set
features = ['sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15']
autoscaler = MinMaxScaler()

x_train1[features] = autoscaler.fit_transform(x_train1[features])
x_test1[features] = autoscaler.fit_transform(x_test1[features])

In [ ]:
x_train1 = x_train1[['bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15', 'month', 'new_grade', 'x0_98002', 'x0_98003', 'x0_98005', 'x0_98007', 'x0_98010', 'x0_98022', 'x0_98023', 'x0_98024', 'x0_98030', 'x0_98031', 'x0_98032', 'x0_98040', 'x0_98042', 'x0_98055', 'x0_98056', 'x0_98058', 'x0_98070', 'x0_98092', 'x0_98108', 'x0_98168', 'x0_98188', 'x0_98198','seattle','kent','bellevue']]
x_test1 = x_test1[['bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15', 'month', 'new_grade', 'x0_98002', 'x0_98003', 'x0_98005', 'x0_98007', 'x0_98010', 'x0_98022', 'x0_98023', 'x0_98024', 'x0_98030', 'x0_98031', 'x0_98032', 'x0_98040', 'x0_98042', 'x0_98055', 'x0_98056', 'x0_98058', 'x0_98070', 'x0_98092', 'x0_98108', 'x0_98168', 'x0_98188', 'x0_98198','seattle','kent','bellevue']]

In [ ]:
import statsmodels.api as sm
y = y_train1
X= x_train1
z = x_test1
Hh = y_test1
Xcont = sm.add_constant(X)

model = sm.OLS(endog = y, exog = Xcont)
model = sm.OLS(y,Xcont)
res = model.fit()
#testmodel = sm.OLS(z,Hh).fit()
#print(testmodel.summary())
print(res.summary())

In [ ]:
ols3 = LinearRegression()
testsmodel = ols3.fit(X,y)
print(testsmodel.score(X,y)) # train
print(testsmodel.score(z,Hh)) # test

Model3 R2 is at 0.65. However, and some outliers is eliminated while R2 score decreases. 

The Training and Test scores are with 2%, so it shows me that the our model is not underfit/overfit.

In [ ]:
plt.scatter(np.exp(model3_predictions), np.exp(y_test1))
plt.xlabel('predicting testing price in $')
plt.ylabel('actual testing sale price in $')
fig.suptitle('prediction price vs actual sale price')

In [ ]:
sns.distplot((y_test1-model3_predictions))

QQ plot: points forming a line that’s roughly straight line- left tail is reducing.

In [ ]:
fig = sm.graphics.qqplot(res.resid, dist=stats.norm, line='45', fit=True)

In [ ]:
plt.scatter(np.exp(model3_predictions),(np.exp(y_test1)-np.exp(model3_predictions)))
plt.axhline(y=0, color='r', linestyle='-')
plt.title("model3 residual plot, testing set residual")
plt.show()

# Prediction with model3

While the R2 score went down to 63%, I decide to go with model with fewer outliers and less predictors.


here is the prediction w the training set and the residual plot

In [ ]:
# make a prediction w the x_train set
model3_train_prediction = ols3.predict(X)


make a plot between x_train prediction vs y_train 

In [ ]:
plt.scatter(np.exp(model3_train_prediction), np.exp(y_train1))
plt.xlabel('predicting testing price in $')
plt.ylabel('actual testing sale price in $')
fig.suptitle('prediction price vs actual sale price')

residual plot of model_3 train set

In [ ]:
plt.scatter(np.exp(model3_train_prediction),(np.exp(y_train1)-np.exp(model3_train_prediction)))
plt.axhline(y=0, color='r', linestyle='-')
plt.title("model3 residual plot, train set residual")
plt.show()

In [ ]:
#find the intercept
ols3.intercept_

In [ ]:
#fint the coefficient
ols3.coefficient_

In [ ]:
#find predicted value with test set
model3_predictions = ols3.predict(z)
model3_predictions

In [ ]:
# compute a dataframe with city and predict and actual price.
df_predict_price = pd.DataFrame(model3_predictions)

In [ ]:
df_predict_price = pd.DataFrame({'seattle':z['seattle'],'kent':z['kent'],'bellevue':z['bellevue'],'actual':np.exp(y_test1),'predicted':np.exp(model3_predictions)})


In [ ]:
df_seattle = df_predict_price.loc[df_predict_price['seattle'] ==1]
df_seattle.describe()


In [ ]:
df_kent = df_predict_price.loc[df_predict_price['kent'] ==1]
df_kent.describe()

In [ ]:
df_bellevue = df_predict_price.loc[df_predict_price['bellevue'] ==1]
df_bellevue.describe()

# Recommendations:


# Next Steps:


While this analysis can help us analyse house sale, there are some additional steps we can take to provide a more detailed analysis.

Investigate with different typr of models such as polynomial models that may fit predictors that may not have a linear relationship with the outcome instead of linear model

Continue to collect data for house sale during covid. Are we starting to see the house sale depends on sqft_living, bedroom and bathroom size since majority of the people are working from home.

Adjust house sale for inflation. We may get a more accurate analysis if house sale is adjusted for inflation. 

Collect additional data for the upcoming year and more. As recession is looming, and the Fed decides to increrase the interest rate, would we able to analyze it and how it impacts the house sale.